# CS598 PSL — Assignment 1

Group Members: Rahul Kasibhatla, Neeyati Devanagondi, Manas Gandhi

# Question 1

In [13]:
# (a) Fit a best subset selection algorithm to the data set and report the best model of each
# model size (up to 8 variables, excluding the intercept) and their prediction errors. Make
# sure that you simplify your output to only present the essential information.

import pandas as pd 
import numpy as np
import itertools
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

test_df = pd.read_csv("mls_test.csv")
train_df = pd.read_csv("mls_train.csv")

y_train = train_df["salary"]
X_train = train_df.drop(columns=["salary"])

y_test = test_df["salary"]
X_test = test_df.drop(columns=["salary"])

results = []

for k in range(1, 9): 
    best_mse = np.inf
    best_vars = None
    
    for combo in itertools.combinations(X_train.columns, k):
        model = LinearRegression()
        model.fit(X_train[list(combo)], y_train)

        y_pred = model.predict(X_test[list(combo)])
        mse = mean_squared_error(y_test, y_pred)

        if mse < best_mse:
            best_mse = mse
            best_vars = combo

    results.append({
        "Size": k,
        "Selected variables": best_vars,
        "Predicition error (mse)": best_mse
    })


results_a = pd.DataFrame(results)
print(results_a.to_string(index=False))

 Size                                                                                 Selected variables  Predicition error (mse)
    1                                                                            (ontarget_scoring_att,)             6.521076e+11
    2                                                                     (sub_on, ontarget_scoring_att)             6.295688e+11
    3                                                              (sub_on, goals, ontarget_scoring_att)             6.243642e+11
    4                                             (height, cm, weight, kg, sub_on, ontarget_scoring_att)             6.186855e+11
    5                                      (height, cm, weight, kg, sub_on, goals, ontarget_scoring_att)             6.131527e+11
    6                          (height, cm, weight, kg, sub_on, goals, aerial_won, ontarget_scoring_att)             6.091227e+11
    7           (height, cm, weight, kg, sub_on, goals, aerial_won, ontarget_scoring_att, 

In [17]:
# (b) Using the models reported in part (a), which is the best model acording to: (i) AIC, (ii)
# BIC, (iii) Cp-Mallows, and (iv) R2a? For each criterion, report the MSE for both training and testing data.

# AIC
best_aic, best_model_aic = np.inf, None
for row in results_a.itertuples(index=False):
    vars_ = list(row[1])
    k = len(vars_)
    lm = LinearRegression().fit(X_train[vars_], y_train)
    yhat_train = lm.predict(X_train[vars_])
    rss_train = np.sum((y_train - yhat_train)**2)
    curr_aic = len(y_train) * np.log(rss_train/len(y_train)) + 2*k
    
    if curr_aic < best_aic:
        best_aic = curr_aic
        best_model_aic = (vars_,
                          mean_squared_error(y_train, yhat_train),
                          mean_squared_error(y_test, lm.predict(X_test[vars_])))

print("According to AIC-- Best Model:", best_model_aic[0], "| Train MSE:", round(best_model_aic[1],3), "| Test MSE:", round(best_model_aic[2],3))


# BIC
best_bic, best_model_bic = np.inf, None
for row in results_a.itertuples(index=False):
    vars_ = list(row[1])
    k = len(vars_)
    lm = LinearRegression().fit(X_train[vars_], y_train)
    yhat_train = lm.predict(X_train[vars_])
    rss_train = np.sum((y_train - yhat_train)**2)
    bic = len(y_train) * np.log(rss_train/len(y_train)) + k*np.log(len(y_train))
    if bic < best_bic:
        best_bic = bic
        best_model_bic = (vars_,
                          mean_squared_error(y_train, yhat_train),
                          mean_squared_error(y_test, lm.predict(X_test[vars_])))

print("According to BIC-- Best Model:", best_model_bic[0], "| Train MSE:", round(best_model_bic[1],3), "| Test MSE:", round(best_model_bic[2],3))




# Cp
full_model = LinearRegression().fit(X_train, y_train)
rss_full = np.sum((y_train - full_model.predict(X_train))**2)
sigma2_hat = rss_full / (len(y_train) - X_train.shape[1] - 1)

best_cp, best_model_cp = np.inf, None
for row in results_a.itertuples(index=False):
    vars_ = list(row[1])
    k = len(vars_)
    lm = LinearRegression().fit(X_train[vars_], y_train)
    yhat_train = lm.predict(X_train[vars_])
    rss_train = np.sum((y_train - yhat_train)**2)
    cp = rss_train/sigma2_hat + 2*k - len(y_train)

    if cp < best_cp:
        best_cp = cp
        best_model_cp = (vars_,
                         mean_squared_error(y_train, yhat_train),
                         mean_squared_error(y_test, lm.predict(X_test[vars_])))

print("According to Cp-- Best Model:", best_model_cp[0],"| Train MSE:", round(best_model_cp[1],3), "| Test MSE:", round(best_model_cp[2],3))




# R2
tss = np.sum((y_train - np.mean(y_train))**2)

best_r2, best_model_r2 = -np.inf, None
for row in results_a.itertuples(index=False):
    vars_ = list(row[1])
    k = len(vars_)
    lm = LinearRegression().fit(X_train[vars_], y_train)
    yhat_train = lm.predict(X_train[vars_])
    rss_train = np.sum((y_train - yhat_train)**2)
    adj_r2 = 1 - (rss_train/(len(y_train)-k-1)) / (tss/(len(y_train)-1))
    if adj_r2 > best_r2:
        best_r2 = adj_r2
        best_model_r2 = (vars_,
                         mean_squared_error(y_train, yhat_train),
                         mean_squared_error(y_test, lm.predict(X_test[vars_])))

print("According to Adj-R2-- Best Model:", best_model_r2[0], "| Train MSE:", round(best_model_r2[1],3), "| Test MSE:", round(best_model_r2[2],3))

According to AIC-- Best Model: ['height, cm', 'weight, kg', 'sub_on', 'goals', 'yellow_card', 'won_tackle', 'aerial_won', 'ontarget_scoring_att'] | Train MSE: 296811761763.0 | Test MSE: 616410698050.296
According to BIC-- Best Model: ['sub_on', 'ontarget_scoring_att'] | Train MSE: 309434842063.795 | Test MSE: 629568772715.585
According to Cp-- Best Model: ['height, cm', 'weight, kg', 'sub_on', 'goals', 'yellow_card', 'won_tackle', 'aerial_won', 'ontarget_scoring_att'] | Train MSE: 296811761763.0 | Test MSE: 616410698050.296
According to Adj-R2-- Best Model: ['height, cm', 'weight, kg', 'sub_on', 'goals', 'yellow_card', 'won_tackle', 'aerial_won', 'ontarget_scoring_att'] | Train MSE: 296811761763.0 | Test MSE: 616410698050.296


In [29]:
# (c) Fit a ridge regression model to predict a player’s salary. Use cross-validation to select
# the best regularization parameter λ.

from sklearn.linear_model import RidgeCV
from sklearn.metrics import mean_squared_error

lambdas = np.logspace(-3, 5, 100)  

ridge_cv = RidgeCV(alphas=lambdas, store_cv_values=True)
ridge_cv.fit(X_train, y_train)

best_lambda = ridge_cv.alpha_

yhat_train = ridge_cv.predict(X_train)
yhat_test = ridge_cv.predict(X_test)
train_mse = mean_squared_error(y_train, yhat_train)
test_mse = mean_squared_error(y_test, yhat_test)

print("Best λ:", best_lambda)
print("Ridge Regression -- Train MSE:", train_mse, "| Test MSE:", test_mse)


Best λ: 3511.1917342151346
Ridge Regression -- Train MSE: 254511446399.33725 | Test MSE: 661842279049.4984


In [33]:
# (d) Fit a lasso regression model on the same data set. Identify which features are shrunk to
# zero.

from sklearn.linear_model import LassoCV
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler


lambdas = np.logspace(-3, 5, 100)  

lasso_cv = make_pipeline(
    StandardScaler(),
    LassoCV(alphas=np.logspace(-3, 5, 100), cv=5, max_iter=10000, random_state=42)
)
lasso_cv.fit(X_train, y_train)

best_lambda = lasso_cv.named_steps['lassocv'].alpha_

yhat_train = lasso_cv.predict(X_train)
yhat_test = lasso_cv.predict(X_test)
train_mse = mean_squared_error(y_train, yhat_train)
test_mse = mean_squared_error(y_test, yhat_test)
coef = lasso_cv.named_steps['lassocv'].coef_
features = X_train.columns
zero_features = [f for f, c in zip(features, coef) if c == 0]

print("Best λ:", best_lambda)
print("Lasso Regression -- Train MSE:", round(train_mse,3),"| Test MSE:", round(test_mse,3))
print("Features shrunk to zero:", zero_features)


Best λ: 6135.907273413176
Lasso Regression -- Train MSE: 250848266779.776 | Test MSE: 642991348132.554
Features shrunk to zero: ['height, cm', 'game_started', 'mins', 'yellow_card']


(e) Compare the performance of the models in (b) vs. ridge vs. lasso using the MSE. Which model would you recommend for predicting MLS player salaries and why?

Based on MSE, I would recommend the subset 6-varaible model from part b with an MSe OF 6.091227 × 10^11.